In [35]:
import io

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from catboost import CatBoostRegressor
import shap
import ipywidgets as widgets
from ipywidgets import interactive

In [50]:
market = pd.read_csv('estaticos_market.csv')
market.drop('Unnamed: 0', axis=1, inplace=True)
market.set_index(market['id'],inplace=True)
market.drop('id', axis=1, inplace=True)

for col in market.columns:
    if market[col].isna().sum()/market[col].size >= 0.9:
        market.drop(col, axis=1, inplace=True)
        
market_obj = market.select_dtypes(exclude = 'number')
for col in market_obj.columns:
    market[col].fillna(value='NA', inplace=True)

market_num = market.select_dtypes(include = 'number')
inputer = SimpleImputer(strategy = 'median')
market[market_num.columns] = inputer.fit_transform(market_num)

cat = CatBoostRegressor(silent=True, iterations=200, max_depth = 5)

def train_classifier(classifier, client_list):
    market_clients = market.copy()
    market_clients['is_client'] = 0
    market_clients.loc[client_list.array, 'is_client'] = 1
    target = market_clients['is_client']
    features = market_clients.drop('is_client', axis=1)
    cat_columns = features.select_dtypes(exclude = 'number').columns
    classifier.fit(features, target, cat_features=cat_columns)
    
def get_predict(classifier, client_list):
    prospects = market.copy()
    prospects['is_client'] = 0
    prospects.loc[client_list.array, 'is_client'] = 1
    prospects = prospects[prospects['is_client'] == 0]
    prospects.drop('is_client', axis=1, inplace=True)
    return classifier.predict(prospects)

def get_shap(classifier, client_list):
    prospects = market.copy()
    prospects['is_client'] = 0
    prospects.loc[client_list.array, 'is_client'] = 1
    prospects = prospects[prospects['is_client'] == 0]
    prospects.drop('is_client', axis=1, inplace=True)
    explainer = shap.TreeExplainer(cat)
    return explainer.shap_values(prospects)

In [51]:
def load_file(b):
    global client_list
    global predict
    global predict_shap
    client_list = pd.read_csv(io.StringIO(load_widget.data[0].decode()), sep=',')['id']
    train_classifier(cat, client_list)
    predict = get_predict(cat, client_list)
    predict_shap = get_shap(cat, client_list)
    

load_widget = widgets.FileUpload(accept='.csv', multiple=False)
display(load_widget)
load_button = widgets.Button(description = 'Load CSV')
display(load_button)
load_button.on_click(load_file)

FileUpload(value={}, accept='.csv', description='Upload')

Button(description='Load CSV', style=ButtonStyle())

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


In [53]:
cat.get_feature_importance(prettified=True)[:30]

,Feature Id,Importances
0,tx_crescimento_12meses,4.532383
1,min_meses_servicos_all,4.387206
2,idade_empresa_anos,4.353612
3,de_ramo,4.236178
4,qt_coligados_comercio,3.692657
5,empsetorcensitariofaixarendapopulacao,3.619905
6,nu_meses_rescencia,3.019198
7,min_meses_servicos,2.751086
8,tx_rotatividade,2.685946
9,meses_ultima_contratacaco,2.674894


In [54]:
lead_strength = pandas.pd()

array([-0.00070346,  0.00109401,  0.00012261,  0.00014551, -0.00047195])